# Feature Engineering

## Libraries

In [2]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data 

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

from sklearn.ensemble import RandomForestRegressor
# Feature Selection Library
from boruta import BorutaPy

## Read Data

For this pipeline, only Training Set will be used.

In [3]:
full_data = get_simplified_data()
full_data = full_data[full_data['Set']=='Train']
y_train = pd.read_csv('Y_train.csv')

In [4]:
full_data.head()

,ID,Time,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
0,1,01/05/2018 01:00,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
1,2,01/05/2018 02:00,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2,3,01/05/2018 03:00,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
3,4,01/05/2018 04:00,WF1,-0.420144,-1.172552,0.689598,-0.961441,285.842832,94.860604,Train
4,5,01/05/2018 05:00,WF1,0.100398,-1.262762,0.290994,-0.294963,285.967452,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [5]:
WF = 'WF1'
data = full_data[full_data['WF']==WF]

## Feature Creation

### Wind Speed Vector

In [6]:
feature_data = data[['ID','Time','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

Changing Reference for Negative Angle:

In [7]:
feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m'] = 360 - feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m']
feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m'] = 360 - feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m']

### Time-Relative Variables

Variables Last Month, Last Week

### Wavelet Transformations (Check)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below. This step is done after the Wind Speed Vector calculation to avoid Negative Values on the features generated, which would cause problems on the Log Transformations done before Feature Selection.

In [8]:
tsfresh_data = pd.DataFrame()
for variable in ['T', 'CLCT', 'Wind Speed 100m','Wind Direction 100m', 'Wind Speed 10m', 'Wind Direction 10m']:
    df_shift, y = make_forecasting_frame(feature_data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction: 100%|██████████| 15/15 [02:25<00:00,  9.69s/it]


Process tsfresh_data to pass column 'Features' to the other columns

## Feature Selection

Here we do the Feature Selection using Borutapy, a Python Implementation of the Famous R Method. For the method we use a Random Forest Regressor.

In [11]:
rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

In [12]:
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [18]:
feat_selector.fit(tsfresh_data.drop('Feature',axis=1), y_train['Production'])

ValueError: Found input variables with inconsistent numbers of samples: [37428, 37375]

In [ ]:
feat_selector.ranking_